<a href="https://colab.research.google.com/github/cepdnaclk/e19-co544-Bitcoin-Cost-Forecast-System/blob/main/Models/Multivariate_LSTM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data

In [ ]:
!pip install azureml-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.7/137.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#!pip install azureml-sdk
import os
from datetime import date
from azureml.core import Workspace, Dataset, Datastore, Run, Model
import numpy as np
import pandas as pd

print("Date of pipeline ML code run is: ", date.today())
run = Run.get_context()

print("setting up the AML workspace")
ws = Workspace(subscription_id='', resource_group='', workspace_name='')
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

blob_datastore_name = ''
container_name = os.getenv("BLOB_CONTAINER", "")
account_name = os.getenv("BLOB_ACCOUNTNAME", "")
account_key = os.getenv("BLOB_ACCOUNT_KEY", "")

datastore = Datastore.get(ws, '')
print("Setting up the blobstore for reading the input files")

run = Run.get_context()
hist = Dataset.Tabular.from_delimited_files(path= [(datastore, "BTC-USD.csv")]).to_pandas_dataframe()
# Making the 'Date' as the index
hist.index = pd.to_datetime(hist.index)

# Drop the "Dividends" column and "Stock Splits" column
#hist.drop(columns = ['Dividends', 'Stock Splits'], inplace = True)

# Print the data
print(hist.head())

Date of pipeline ML code run is:  2024-06-24
setting up the AML workspace
Performing interactive authentication. Please follow the instructions on the terminal.


Interactive authentication successfully completed.
Bit_Predictor_mlops
Machine_Learning
westus
43a74c73-f7cb-4ce8-b74c-77b08ff7bf47
Setting up the blobstore for reading the input files
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
                                    Date        Open        High         Low  \
1970-01-01 00:00:00.000000000 2014-09-17  465.864014  468.174011  452.421997   
1970-01-01 00:00:00.000000001 2014-09-18  456.859985  456.859985  413.104004   
1970-01-01 00:00:00.000000002 2014-09-19  424.102997  427.834991  384.532013   
1970-01-01 00:00:00.000000003 2014-09-20  394.673004  423.295990  389.882996   
1970-01-01 00:00:00.000000004 2014-09-21  408.084991  412.425995  393.181000   

                                    Close   Adj Close    Volume  
1970-01-01 00:00:00.000000000  457.334015  457.334015  21056800  
1970-01-01 00:00:00.000000001  424.

# Set Inputs and Outputs

In [ ]:
X, y = hist.drop(columns = ['Close', 'Date', 'Adj Close']), hist.Close.values
X.shape, y.shape

((3566, 4), (3566,))

## Standardize Features

In [ ]:
# Import the StandardScaler and MinMaxScaler classes from the sklearn.preprocessing module
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import joblib

# Create an instance of thee MinMaxScaler, which scales the data to a specified range (default is 0 to 1)
mm = MinMaxScaler()

# Creating an instance of the standardScaler, which standardizes the data by removing the mean and scaling to unit variance
ss = StandardScaler()

# Applying the StandardScaler to the feature matrix X to standardize the features
# fit_transform() first fits the scaler to the data (calculating the mean and standard deviation) and then transforms the data
X_trans = ss.fit_transform(X)

# Reshaping the target variable y to be a 2D array with one column, as required by the MinMaxScaler
# fit_transform() first fits the scaler to the data (calculating the min and max values) and then transforms the data
y_trans = mm.fit_transform(y.reshape(-1, 1))

standard_scaler_path = "standard_scaler.pkl"
minmax_scaler_path = "minmax_scaler.pkl"

# Save StandardScaler (ss)
joblib.dump(ss, standard_scaler_path)

# Save MinMaxScaler (mm)
joblib.dump(mm, minmax_scaler_path)

['minmax_scaler.pkl']

## Split a multivariate sequence past, future samples (X abd y)

In [ ]:
import numpy as np

# split a multivariate sequence past, future samples (X and y)
def split_sequences(input_sequences, output_sequence, n_steps_in, n_steps_out):

    X, y = list(), list() # instantiate X and y

    for i in range(len(input_sequences)):

        # find the end of the input, output sequence
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out - 1

        # check if we are beyond the dataset
        if out_end_ix > len(input_sequences): break

        # gather input and output of the pattern
        seq_x, seq_y = input_sequences[i:end_ix], output_sequence[end_ix-1:out_end_ix, -1]
        X.append(seq_x), y.append(seq_y)

    return np.array(X), np.array(y)

X_ss, y_mm = split_sequences(X_trans, y_trans, 2, 1)
print(X_ss.shape, y_mm.shape)

(3565, 2, 4) (3565, 1)


## Check the y_mm sample

In [ ]:
print("y_mm[0]:", y_mm[0])
print("y_trans[99:100].squeeze(1):", y_trans[99:100].squeeze(1))

assert y_mm[0].all() == y_trans[99:100].squeeze(1).all()

y_mm[0]

y_mm[0]: [0.00337886]
y_trans[99:100].squeeze(1): [0.00193271]


array([0.00337886])

In [ ]:
y_trans[99:100].squeeze(1)

array([0.00193271])

## Split Training & Test Sets

In [ ]:
total_samples = len(X)
train_test_cutoff = round(0.90 * total_samples)

X_train = X_ss[:-101]
X_test = X_ss[-101:]

y_train = y_mm[:-101]
y_test = y_mm[-101:]

print("Training Shape:", X_train.shape, y_train.shape)
print("Testing Shape:", X_test.shape, y_test.shape)

Training Shape: (3464, 2, 4) (3464, 1)
Testing Shape: (101, 2, 4) (101, 1)


## Convert Data Into Tensors

In [ ]:
import torch

# convert to pytorch tensors
X_train_tensors = torch.Tensor(X_train).requires_grad_(True)
X_test_tensors = torch.Tensor(X_test).requires_grad_(True)

y_train_tensors = torch.Tensor(y_train).requires_grad_(True)
y_test_tensors = torch.Tensor(y_test).requires_grad_(True)


## Reshaping to rows, timestamps, features

In [ ]:
# reshaping to rows, timestamps, features
X_train_tensors_final = torch.reshape(X_train_tensors,
                                      (X_train_tensors.shape[0], 2,
                                       X_train_tensors.shape[2]))
X_test_tensors_final = torch.reshape(X_test_tensors,
                                     (X_test_tensors.shape[0], 2,
                                      X_test_tensors.shape[2]))

print("Training Shape:", X_train_tensors_final.shape, y_train_tensors.shape)
print("Testing Shape:", X_test_tensors_final.shape, y_test_tensors.shape)

Training Shape: torch.Size([3464, 2, 4]) torch.Size([3464, 1])
Testing Shape: torch.Size([101, 2, 4]) torch.Size([101, 1])


In [ ]:
X_check, y_check = split_sequences(X, y.reshape(-1, 1), 2, 1)
X_check[-1][0:4]

X.iloc[-149:-145]

,Open,High,Low,Volume
1970-01-01 00:00:00.000003417,40075.550781,40254.480469,39545.664063,18491782013
1970-01-01 00:00:00.000003418,39936.816406,42209.386719,39825.691406,25598119893
1970-01-01 00:00:00.000003419,41815.625000,42195.632813,41431.281250,11422941934
1970-01-01 00:00:00.000003420,42126.125000,42797.175781,41696.910156,16858971687


In [ ]:
y_check[-1]

array([63727.816406])

In [ ]:
hist.Close.values[-1:]

array([63727.816406])

# LSTM Model

In [ ]:
import torch.nn as nn

class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super().__init__()
        self.num_classes = num_classes # output size
        self.num_layers = num_layers # number of recurrent layers in the lstm
        self.input_size = input_size # input size
        self.hidden_size = hidden_size # neurons in each lstm layer
        # LSTM model
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True, dropout=0.2) # lstm
        self.fc_1 =  nn.Linear(hidden_size, 128) # fully connected
        self.fc_2 = nn.Linear(128, num_classes) # fully connected last layer
        self.relu = nn.ReLU()

    def forward(self,x):
        # hidden state
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        # cell state
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        # propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) # (input, hidden, and internal state)
        hn = hn.view(-1, self.hidden_size) # reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) # first dense
        out = self.relu(out) # relu
        out = self.fc_2(out) # final output
        return out

# Training

In [ ]:
def training_loop(n_epochs, lstm, optimiser, loss_fn, X_train, y_train,
                  X_test, y_test):
    for epoch in range(n_epochs):
        lstm.train()
        outputs = lstm.forward(X_train) # forward pass
        optimiser.zero_grad() # calculate the gradient, manually setting to 0
        # obtain the loss function
        loss = loss_fn(outputs, y_train)
        loss.backward() # calculates the loss of the loss function
        optimiser.step() # improve from loss, i.e backprop
        # test loss
        lstm.eval()
        test_preds = lstm(X_test)
        test_loss = loss_fn(test_preds, y_test)
        if epoch % 100 == 0:
            print("Epoch: %d, train loss: %1.5f, test loss: %1.5f" % (epoch,
                                                                      loss.item(),
                                                                      test_loss.item()))

# Instance of a LSTM Model

In [ ]:
import warnings
warnings.filterwarnings('ignore')

n_epochs = 1000 # 1000 epochs
learning_rate = 0.001 # 0.001 lr

input_size = 4 # number of features
hidden_size = 2 # number of features in hidden state
num_layers = 1 # number of stacked lstm layers

num_classes = 1 # number of output classes

lstm = LSTM(num_classes,
              input_size,
              hidden_size,
              num_layers)

# Training Loop (MSE as the loss function)

In [ ]:
loss_fn = torch.nn.MSELoss()    # mean-squared error for regression
optimiser = torch.optim.Adam(lstm.parameters(), lr=learning_rate)


training_loop(n_epochs=n_epochs,
              lstm=lstm,
              optimiser=optimiser,
              loss_fn=loss_fn,
              X_train=X_train_tensors_final,
              y_train=y_train_tensors,
              X_test=X_test_tensors_final,
              y_test=y_test_tensors)



Epoch: 0, train loss: 0.32465, test loss: 1.35216
Epoch: 100, train loss: 0.03357, test loss: 0.28098
Epoch: 200, train loss: 0.00288, test loss: 0.05940
Epoch: 300, train loss: 0.00188, test loss: 0.04825
Epoch: 400, train loss: 0.00136, test loss: 0.03809
Epoch: 500, train loss: 0.00088, test loss: 0.02833
Epoch: 600, train loss: 0.00071, test loss: 0.02322
Epoch: 700, train loss: 0.00059, test loss: 0.01971
Epoch: 800, train loss: 0.00051, test loss: 0.01716
Epoch: 900, train loss: 0.00043, test loss: 0.01483


In [ ]:
input_data = [
    [465.864014, 468.174011, 452.421997, 21056800],  # Day 1
    [456.859985, 456.859985, 413.104004, 34483200]]  # Day 2

  # Assuming 'scaler' is already fitted to the training data
input_data_scaled = ss.transform(input_data)  # Scale the data
input_tensor = torch.Tensor(input_data_scaled).unsqueeze(0)  # Convert to tensor and add batch dimension

# Make prediction with the LSTM model
lstm.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient computation
    prediction_tensor = lstm(input_tensor)  # Make the prediction

# Reshape the prediction to match the number of features
prediction_reshaped = prediction_tensor.numpy().reshape(-1, 1)

# Reverse the scaling of the prediction
prediction_reshaped = mm.inverse_transform(prediction_reshaped)

# Since we're only interested in the 'Close' price, select the relevant column
predicted_close_price = prediction_reshaped[0][0]

print(f"Predicted Close price: {predicted_close_price}")


Predicted Close price: 611.7787475585938


In [ ]:
import torch
import joblib
import numpy as np
from azureml.core.model import Model

# Load the model and scalers from the registered model
def init():
    global model
    global ss
    global mm
    # Replace 'model_name' with the actual name of the model registered in Azure ML
    model_path = Model.get_model_path('L')
    model = joblib.load(model_path + '/lstm_model.pkl')
    ss = joblib.load(model_path + '/standard_scaler.pkl')
    mm = joblib.load(model_path + '/minmax_scaler.pkl')

# Run the model to make predictions
def run(raw_data):
    try:
        # Convert raw data into a format suitable for prediction
        data = np.array(raw_data)
        data = ss.transform(data)  # Scale the data
        data = torch.Tensor(data).unsqueeze(0)  # Convert to tensor and add batch dimension

        # Make prediction
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():  # Disable gradient computation
            prediction_tensor = model(data)  # Make the prediction

        # Reshape the prediction to match the number of features
        prediction_reshaped = prediction_tensor.numpy().reshape(-1, 1)

        # Reverse the scaling of the prediction
        prediction = mm.inverse_transform(prediction_reshaped)

        # Return the prediction
        return prediction[0][0]
    except Exception as e:
        error = str(e)
        return error


# Prediction

In [ ]:
# import matplotlib.pyplot as plt

# df_X_ss = ss.transform(hist.drop(columns=['Close'])) # old transformers
# df_y_mm = mm.transform(hist.Close.values.reshape(-1, 1)) # old transformers
# # split the sequence
# df_X_ss, df_y_mm = split_sequences(df_X_ss, df_y_mm, 100, 1)
# # converting to tensors
# df_X_ss = torch.Tensor(df_X_ss)
# df_y_mm = torch.Tensor(df_y_mm)
# # reshaping the dataset
# df_X_ss = torch.reshape(df_X_ss, (df_X_ss.shape[0], 100, df_X_ss.shape[2]))

# train_predict = lstm(df_X_ss) # forward pass
# data_predict = train_predict.data.numpy() # numpy conversion
# dataY_plot = df_y_mm.data.numpy()

# data_predict = mm.inverse_transform(data_predict) # reverse transformation

# dataY_plot = mm.inverse_transform(dataY_plot)
# true, preds = [], []
# for i in range(len(dataY_plot)):
#     true.append(dataY_plot[i][0])
# for i in range(len(data_predict)):
#     preds.append(data_predict[i][0])


# plt.figure(figsize=(10,6)) #plotting
# plt.axvline(x=train_test_cutoff, c='r', linestyle='--') # size of the training set

# plt.plot(true, label='Actual Data') # actual plot
# plt.plot(preds, label='Predicted Data') # predicted plot
# plt.title('Time-Series Prediction')
# plt.legend()
# plt.savefig("whole_plot.png", dpi=300)
# plt.show()

# print(preds[-1])

# Save as a pickle file

In [ ]:
import pickle

# Save the LSTM model
with open('lstm_model.pkl', 'wb') as f:
    pickle.dump(lstm, f)

In [ ]:
loaded_model = pickle.load(open('lstm_model.pkl', 'rb'))

In [ ]:
input_data = [
    [465.864014, 468.174011, 452.421997, 21056800],  # Day 1
    [456.859985, 456.859985, 413.104004, 34483200]]  # Day 2

  # Assuming 'scaler' is already fitted to the training data
input_data_scaled = ss.transform(input_data)  # Scale the data
input_tensor = torch.Tensor(input_data_scaled).unsqueeze(0)  # Convert to tensor and add batch dimension

# Make prediction with the LSTM model
loaded_model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient computation
    prediction_tensor = loaded_model(input_tensor)  # Make the prediction

# Reshape the prediction to match the number of features
prediction_reshaped = prediction_tensor.numpy().reshape(-1, 1)

# Reverse the scaling of the prediction
prediction_reshaped = mm.inverse_transform(prediction_reshaped)

# Since we're only interested in the 'Close' price, select the relevant column
predicted_close_price = prediction_reshaped[0][0]

print(f"Predicted Close price: {predicted_close_price}")


Predicted Close price: 611.7787475585938


# Model Evaluation

## RMSE

In [ ]:
import torch

# Function to evaluate the model
def evaluate_model(model, X_test, y_test):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Turn off gradients for validation, saves memory and computations
        predictions = model(X_test)
        # Calculate the RMSE loss
        mse = torch.mean((predictions - y_test)**2)  # Calculate mean squared error
        rmse = torch.sqrt(mse)  # Calculate the square root of the MSE to get RMSE
    return rmse.item()  # Return RMSE value

# Call the evaluate_model function
test_rmse = evaluate_model(lstm, X_test_tensors_final, y_test_tensors)
print(f"Test RMSE: {test_rmse}")


Test RMSE: 0.022388571873307228


## Test Loss

In [ ]:
import torch

# Function to evaluate the model
def evaluate_model(model, X_test, y_test):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Turn off gradients for validation, saves memory and computations
        predictions = model(X_test)
        # Calculate the loss
        loss_fn = torch.nn.MSELoss()
        loss = loss_fn(predictions, y_test)
    return loss.item()

# Call the evaluate_model function
test_loss = evaluate_model(lstm, X_test_tensors_final, y_test_tensors)
print(f"Test Loss: {test_loss}")


Test Loss: 0.0005012481124140322


## MAE

In [ ]:
import torch
from torch.nn.functional import l1_loss

# 'predictions' and 'true_values' are PyTorch tensors of the same shape
predictions = lstm(X_test_tensors_final)
true_values = y_test_tensors

# Calculate MAE
mae = l1_loss(predictions, true_values, reduction='mean').item()
print(f"Mean Absolute Error: {mae}")



Mean Absolute Error: 0.017877081409096718


## R-squared

In [ ]:
from sklearn.metrics import r2_score

# lstm and X_test_tensors_final are already defined and lstm is trained
lstm.eval()  # Set the model to evaluation mode
y_pred = lstm(X_test_tensors_final)
y_pred_numpy = y_pred.data.numpy()

# y_test_tensors is the actual values tensor and mm is the MinMaxScaler
y_test_numpy = y_test_tensors.data.numpy()

# Inverse transform the scaled data to original scale
y_pred_rescaled = mm.inverse_transform(y_pred_numpy)
y_test_rescaled = mm.inverse_transform(y_test_numpy)

# Calculate R-squared
r_squared = r2_score(y_test_rescaled, y_pred_rescaled)
print(f"R-squared: {r_squared}")




R-squared: 0.902178113624038


## MAPE

In [ ]:
import torch

# Function to calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred, eps=1e-8):
    # Avoid division by zero
    y_true, y_pred = torch.tensor(y_true), torch.tensor(y_pred)
    mape = torch.mean(torch.abs((y_true - y_pred) / torch.clamp(y_true, min=eps)))
    return mape

# Function to evaluate the model
def evaluate_model(model, X_test, y_test):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Turn off gradients for validation, saves memory and computations
        predictions = model(X_test)
        # Calculate the MAPE
        mape = mean_absolute_percentage_error(y_test, predictions)
    return mape.item()

# Call the evaluate_model function
test_mape = evaluate_model(lstm, X_test_tensors_final, y_test_tensors)
print(f"Test MAPE: {test_mape}")


Test MAPE: 0.019540103152394295
